# 0.4_matrix

In [153]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext blackcellmagic

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

def nrows(n):
    pd.options.display.max_rows = n

nrows(5)

from coding_the_matrix import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


In [2]:
A = listlist2mat([[10, 20, 30, 40], [50, 60, 70, 80]])

In [4]:
print(A)

    0   1   2   3
0  10  20  30  40
1  50  60  70  80


In [5]:
def mat2Vec(M):
    R, C = M.D
    return Vec({(r,c) for r in R for c in C}, M.f) 

In [6]:
def tranpose(M):
    R, C = M.D
    D = (C, R)
    f = {(c, r): v for (r, c), v in M.f.items()}
    return Mat(D, f)

In [7]:
print(tranpose(A))

    0   1
0  10  50
1  20  60
2  30  70
3  40  80


In [39]:
def button_vectors(n):
    D = {(x, y) for x, y in itertools.product(range(n), range(n))}
    vecdict = {
        (i, j): Vec(
            D,
            dict([((x, j), one) for x in range(max(i - 1, 0), min(i + 2, n))]) | 
            dict([((i, y), one) for y in range(max(j - 1, 0), min(j + 2, n))]),
        )
        for (i, j) in D
    }
    return vecdict

In [78]:
value(vectors)

Vec({(4, 0), (3, 4), (4, 3), (3, 1), (0, 2), (2, 2), (1, 0), (1, 3), (4, 2), (3, 0), (3, 3), (0, 1), (2, 4), (1, 2), (0, 4), (2, 1), (3, 2), (4, 1), (4, 4), (0, 0), (1, 1), (0, 3), (2, 0), (1, 4), (2, 3)}, {(3, 0): 1, (4, 0): 1, (4, 1): 1})

In [67]:
keys(vectors)

dict_keys([(4, 0), (3, 4), (4, 3), (3, 1), (0, 2), (2, 2), (1, 0), (1, 3), (4, 2), (3, 0), (3, 3), (0, 1), (2, 4), (1, 2), (0, 4), (2, 1), (3, 2), (4, 1), (4, 4), (0, 0), (1, 1), (0, 3), (2, 0), (1, 4), (2, 3)])

In [49]:
vectors = button_vectors(5)

In [51]:
len(vectors)

25

In [59]:
b = vectors[(0, 0)]

In [40]:
dict([(4, 3)]) | dict([(4, 1)])

{4: 1}

In [42]:
B = coldict2mat(button_vectors(5))

In [60]:
s = Vec(b.D, {(2, 2): one})

## Gnome

In [82]:
D = {'metal', 'concrete', 'plastic', 'water', 'electricity'}
v_gnome = Vec(D, {'concrete': 1.3, 'plastic': .2, 'water': .8, 'electricity': .4})
v_hoop = Vec(D, {'plastic': 1.5, 'water': .4, 'electricity': .3})
v_slinky = Vec(D, {'metal': .25, 'water': .2, 'electricity': .7})
v_putty = Vec(D, {'plastic': .3, 'water': .7, 'electricity': .5})
v_shooter = Vec(D, {'metal': .15, 'plastic': .5, 'water': .4, 'electricity': .8})

In [83]:
240*v_gnome + 55 * v_hoop

Vec({'electricity', 'metal', 'plastic', 'concrete', 'water'}, {'electricity': 112.5, 'plastic': 130.5, 'concrete': 312.0, 'water': 214.0})

In [145]:
rowdict = {
    'gnome': v_gnome,
    'hoop': v_hoop,
    'slinky': v_slinky,
    'putty': v_putty,
    'shooter': v_shooter,
}

In [146]:
M = rowdict2mat(rowdict)

In [147]:
print(M)

         electricity  metal  plastic  concrete  water
gnome            0.4   0.00      0.2       1.3    0.8
hoop             0.3   0.00      1.5       0.0    0.4
shooter          0.8   0.15      0.5       0.0    0.4
slinky           0.7   0.25      0.0       0.0    0.2
putty            0.5   0.00      0.3       0.0    0.7


In [148]:
R = set(rowdict.keys())

In [149]:
u = Vec(R, {'putty': 133, 'gnome': 240, 'slinky': 150, 'hoop': 55, 'shooter': 90})

In [150]:
M.D

({'gnome', 'hoop', 'putty', 'shooter', 'slinky'},
 {'concrete', 'electricity', 'metal', 'plastic', 'water'})

Vec(M.D[1], {k: u * vec for k, vec in mat2coldict(M).items()})

In [151]:
u.D

{'gnome', 'hoop', 'putty', 'shooter', 'slinky'}

In [152]:
u * M

TypeError: unsupported operand type(s) for *: 'int' and 'Mat'

In [3]:
A.D

({0, 1}, {0, 1, 2, 3})

In [5]:
A.f.keys()

dict_keys([(0, 0), (0, 1), (0, 2), (0, 3), (1, 0), (1, 1), (1, 2), (1, 3)])

In [14]:
import itertools
for r, c in itertools.product(*D):
    print(f"('{r}', '{c}'): ,")

('a', '@'): ,
('a', '#'): ,
('a', '?'): ,
('b', '@'): ,
('b', '#'): ,
('b', '?'): ,


In [7]:
D = ({'a', 'b'}, {'@', '#', '?'})

In [12]:
f = {
    ("a", "@"): 1,
    ("a", "#"): 2,
    ("a", "?"): 3,
    ("b", "@"): 10,
    ("b", "#"): 20,
    ("b", "?"): 30,
}

In [13]:
M = Mat(D, f)

In [31]:
def mat2rowdict(M):
    """return the rowdict representation of M"""
    R, C = M.D
    return {r: Vec(C, {c: M.f[(r, c)] for c in C}) for r in R}

In [32]:
mat2rowdict(M)

{'a': Vec({'@', '#', '?'}, {'@': 1, '#': 2, '?': 3}),
 'b': Vec({'@', '#', '?'}, {'@': 10, '#': 20, '?': 30})}

In [33]:
def mat2coldict(M):
    """return the coldict representation of M"""
    R, C = M.D
    return {c: Vec(R, {r: M.f[(r, c)] for r in R}) for c in C}

In [34]:
mat2coldict(M)

{'@': Vec({'a', 'b'}, {'a': 1, 'b': 10}),
 '#': Vec({'a', 'b'}, {'a': 2, 'b': 20}),
 '?': Vec({'a', 'b'}, {'a': 3, 'b': 30})}